In [719]:
# Importing necessary Libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 

In [735]:
#____________ Choose the Images Dataset numer to stitching the images _____________#
#######################
Image_Dataset_No = 1
# <-----
#######################

#____________ Choose if want to use in-built functions or not  _____________#
#######################
in_built = True       # <-----
#######################

flag = ""
if(in_built==True):
    flag = "_in_built"

## Helper Functions:
#### A) Detect and extract key features from a given image:

In [736]:
def detect_and_extract_features(I):
    #_____ Using SIFT Algorithm from the in-built cv2 library
    sift_algo = cv2.xfeatures2d.SIFT_create()
    features, f_descriptors = sift_algo.detectAndCompute(I, None)
    return features, f_descriptors

#### B) Feature Matching [Simple Brute Force Method] 

In [746]:
def match_features(f1, f2, d1, d2, I1, I2):
    #_____ Using Brute Force Matching Algo from the in-built cv2 library that matches the features from the given two images
    matching_algo = cv2.BFMatcher(cv2.NORM_L2, True)
    matchings = matching_algo.match(d1, d2)
#     matchings_sort = sorted(matchings, key = lambda x:x.distance)
#     match_I = cv2.drawMatches(I1,f1,I2,f2,matchings_sort[:50],outImg = None, flags=2)
#     cv2.imwrite('Matches_I1_I2.png', match_I)
    return matchings

#### C) Homography Matrix Calculation using Best Point Correspondences

In [723]:
def find_homography(point_correspondences):
    #___ Creating a transformation matrix that will stitch the given 3 images based on their point_correspondences
    # Implementation of the Direct Linear Transformation
    A = [] # Assemble
    for points in point_correspondences:
        x1, y1, x2, y2 = points
        A.append([x1, y1, 1, 0, 0, 0, -x2*x1, -x2*y1, -x2])
        A.append([0, 0, 0, x1, y1, 1, -y2*x1, -y2*y1, -y2])        
    A = np.asarray(A)
    U, E, Vt = np.linalg.svd(A) # Using Singular Value Decomposition to find the tranformation vector [Last column of Vt]
    H = (Vt[-1,:] / Vt[-1,-1]).reshape(3, 3) # Normalization
    return H # 3*3 Homography Transformation Matrix

#### D) Using RANSAC (RANdom SAmple Consensus) Algorithm that best fits the Feature matchings of two images
- Focuses largely on the Inliners

In [724]:
def ransac_algo(point_correspondences, threshold, N=500):
    maxInliers = [] # Stores the maximum Inliers
    Homography_matrix = None
    
    # Run this for multiple iterations to find the best line fit that has maximum no. of inliers
    for i in range(N):
        # Randomly select 4 points to estimate a fit line
        indices = np.random.choice(range(len(point_correspondences)),4, replace = False)
        points = map(point_correspondences.__getitem__, indices) 
        H = find_homography(points)
        inliers = [] # To calculate the no. of inliners using this fit
        for points in point_correspondences:
            x1, y1, x2, y2 = points
            vector1 = np.asarray([x1, y1, 1]).T
            transformed_vector = H.dot(vector1) # Find the tranformed vector
            transformed_vector /= transformed_vector[-1] #Normalization
            vector2 = np.asarray([x2,y2,1])
            # Inlier if the point lies very close inside the threshold area.
            if(np.linalg.norm(transformed_vector-vector2)<=threshold):
                inliers.append(points)
        if(len(inliers) > len(maxInliers)):
            maxInliers = inliers
            Homography_matrix = H
    # return the best estimated homography matrix
    return Homography_matrix, maxInliers

## Main Function:
#### Process:
    - Detect and Extract Features from both the Images
    - Feature Matching and Homography Matrix estimation using RANSAC

In [725]:
def main(I1,I2, in_built):
    f1,d1 = detect_and_extract_features(I1) # For image-1
#     I1_dash = cv2.drawKeypoints(I1, f1,outImage = None)
#     cv2.imwrite('sift_I1.png', I1_dash)
    print("No. of features extracted using SIFT from Ia: {}".format(len(f1)))
    f2,d2 = detect_and_extract_features(I2) # For image-2
#     I2_dash = cv2.drawKeypoints(I2, f2,outImage = None)
#     cv2.imwrite('sift_I2.png', I2_dash)
    print("No. of features extracted using SIFT from Ib: {}".format(len(f2)))

    matchings = match_features(f1, f2, d1, d2, I1, I2)
    print("No. of features matched using individual SIFT descriptors: {}".format(len(matchings)))
    point_correspondences = []
    pcAs = []
    pcBs = []
    for match in matchings:
        x1, y1 = f1[match.queryIdx].pt
        x2, y2 = f2[match.trainIdx].pt
        pcAs.append(f1[match.queryIdx].pt)
        pcBs.append(f2[match.trainIdx].pt)
        point_correspondences.append([x1, y1, x2, y2])

    threshold = 5
    
    ### Without the use of inbuilt functions
    if(in_built == False):
        H, maxx = ransac_algo(point_correspondences,threshold,800)
        print("Max Inliers", len(maxx))
    else:### Homography estimation using inbuilt functions
        H, status = cv2.findHomography( np.matrix(pcAs), np.matrix(pcBs), cv2.RANSAC, threshold)

#     print("Homography Matrix:", H)
    
    return H

In [747]:
# Just for Matching_Example
# I2 = cv2.imread("./2_2.jpg")
# I1 = cv2.imread("./2_3.jpg")
# H_12 = main(I1,I2, in_built)

No. of features extracted using SIFT from Ia: 1718
No. of features extracted using SIFT from Ib: 1447
No. of features matched using individual SIFT descriptors: 538


### Loading the Datasets and Calculating all the requires Homography matricies for Stitching 4 images

In [726]:
# I1 --> Right-most Image
# I4 --> left-most Image

# I1 Dataset
if(Image_Dataset_No == 1):
    I4 = cv2.imread("./Data/I1/STB_0032.jpg")
    I3 = cv2.imread("./Data/I1/STC_0033.jpg")
    I2 = cv2.imread("./Data/I1/STD_0034.jpg")
    I1 = cv2.imread("./Data/I1/STE_0035.jpg")

# I2 Dataset
if(Image_Dataset_No == 2):
    I4 = cv2.imread("./Data/I2/2_1.jpg")
    I3 = cv2.imread("./Data/I2/2_2.jpg")
    I2 = cv2.imread("./Data/I2/2_3.jpg")
    I1 = cv2.imread("./Data/I2/2_4.jpg")

# I3 Dataset
if(Image_Dataset_No == 3):
    I4 = cv2.imread("./Data/I3/3_2.jpg")
    I3 = cv2.imread("./Data/I3/3_3.jpg")
    I2 = cv2.imread("./Data/I3/3_4.jpg")
    I1 = cv2.imread("./Data/I3/3_5.jpg")

# I4 Dataset
if(Image_Dataset_No == 4):
    I4 = cv2.imread("./Data/I4/DSC02931.jpg")
    I3 = cv2.imread("./Data/I4/DSC02932.jpg")
    I2 = cv2.imread("./Data/I4/DSC02933.jpg")
    I1 = cv2.imread("./Data/I4/DSC02934.jpg")

# I5 Dataset
if(Image_Dataset_No == 5):
    I4 = cv2.imread("./Data/I5/DSC03002.jpg")
    I3 = cv2.imread("./Data/I5/DSC03003.jpg")
    I2 = cv2.imread("./Data/I5/DSC03004.jpg")
    I1 = cv2.imread("./Data/I5/DSC03005.jpg")

# I6 Dataset
if(Image_Dataset_No == 6):
    I4 = cv2.imread("./Data/I6/1_1.jpg")
    I3 = cv2.imread("./Data/I6/1_2.jpg")
    I2 = cv2.imread("./Data/I6/1_4.jpg")
    I1 = cv2.imread("./Data/I6/1_3.jpg")

I1 = cv2.resize(I1,(800,512))
I2 = cv2.resize(I2,(800,512))
I3 = cv2.resize(I3,(800,512))
I4 = cv2.resize(I4,(800,512))

# The  main function here does the feature detection, extraction and returns a generated homography matrix
# For stitching 4 images we use the following homographies to stitch the images and build the panaroma!
H_12 = main(I1,I2, in_built) # Homography of I1 with respect to I2
H_23 = main(I2,I3, in_built) # Homography of I2 with respect to I3
H_34 = main(I3,I4, in_built) # Homography of I3 with respect to I4

No. of features extracted using SIFT from Ia: 1823
No. of features extracted using SIFT from Ib: 2234
No. of features matched using individual SIFT descriptors: 758
Max Inliers 411
No. of features extracted using SIFT from Ia: 2234
No. of features extracted using SIFT from Ib: 2970
No. of features matched using individual SIFT descriptors: 881
Max Inliers 174
No. of features extracted using SIFT from Ia: 2970
No. of features extracted using SIFT from Ib: 2817
No. of features matched using individual SIFT descriptors: 1085
Max Inliers 366


### Helper Functions for Blending [Pyramid Method]
#### References:
- http://pages.cs.wisc.edu/~csverma/CS766_09/ImageMosaic/imagemosaic.html
-  Brown, Matthew, and David G. Lowe. ”Automatic panoramic image stitching using invariant features.”
International journal of computer vision 74.1 (2007): 59-73.
- https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_pyramids/py_pyramids.html

In [727]:
#____________ Gaussian Pyramids Genreation _____________#
# It downsizes the image into different levels 
def gaussian_pyr(I, n_levels):
    GPs = [I]
    for i in range(n_levels - 1):
        GPs.append(cv2.pyrDown(GPs[i]))
    return GPs

In [728]:
#____________ Laplacian Pyramids Genreation _____________#
# Downsizing the image into different levels using Gaussian and then 
# expanding the Gaussian into the lower level and subtracting from the image in that level to acquire the Laplacian image
def laplacian_pyr(I, n_levels):
    LPs = []
    for i in range(n_levels - 1):
        I_next = cv2.pyrDown(I)
        LPs.append(I - cv2.pyrUp(I_next, I.shape[1::-1]))
        I = I_next
    LPs.append(I)
    return LPs

In [729]:
#____________ Blending the laplacian pyramids at each level using the mask pyramids at each level _____________#
def blending_pyrs(LPs_I1, LPs_I2, mask_pyrs):
    blended = []
    # At each level, the mask indicates the boundary for blending [Necessary because the images are resizing
    # at each leavel and so is the location of the boundary]
    for i, mask in enumerate(mask_pyrs):
        blended.append(LPs_I1[i]*mask + LPs_I2[i]*(1.0-mask))
    return blended

In [730]:
# _________ Reconstructing the image by expanding the each level laplacians and adding them to the final image __________#
def image_reconstruction(LPs):
    I_new = LPs[-1]
    for I_level in LPs[-2::-1]:
        I_new = cv2.pyrUp(I_new, I_level.shape[1::-1])
        I_new += I_level
    I_new[I_new > 255] = 255
    I_new[I_new < 0] = 0
    return I_new

### WARPING AND BLENDING:
#### Process:
- Create a template for the final stitched image
- Set the Base image into the template [Using the 2nd image of the Panaroma as the base]
- Generate the warp of the left-most image
- Stitch the warped left-most image to the template, meanwhile using the Pyramid blending for better results 

In [731]:
# Template frame Sizes
h_max = (I1.shape[0] + I2.shape[0] + I3.shape[0] + I4.shape[0])
w_max = (I1.shape[1] + I2.shape[1] + I3.shape[1] + I4.shape[1])*2
stitched_image = np.zeros((h_max,w_max,3))
# Offset for the Base Image
h_offset = h_max//2 - I3.shape[0]//2
w_offset = w_max//3 - I3.shape[1]//2
## Base Image set in the to be stitched_image
stitched_image[h_offset : h_offset+I3.shape[0], w_offset : w_offset+I3.shape[1], :] = I3

In [732]:
# Stitching I4 to I3: [Leftmost Image to the base image]

temp_image = np.zeros((h_max,w_max,3)) # To store the Warped I4 Image
common_area_xs = [] # To find the width and position of the intersecting region

x_min = np.min(np.where(stitched_image != 0)[1]) # Minimum location of the already stitched image

#_________ The commented method below is to find the Warped image I4 that can be stitched to I3 easily.
#_________ But this method isn't effective because it finds the transformed I4 using the H34 homography and does not
#_________ necessarily have integral pixel locations. Thus blank dots found in the output Image.

################-----------------------##################
# for y in range(I4.shape[0]):
#     for x in range(I4.shape[1]):
#         v_dash = np.linalg.inv(H_34).dot([x,y,1])
#         v_dash /= v_dash[2]
#         x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
        
#         temp_image[y_dash+h_offset-1 : y_dash+h_offset+2, x_dash+w_offset-1 : x_dash+w_offset+2,:] = I4[y,x,:]
        
#         if(x_dash+w_offset > x_min):
#             common_area_xs.append(x_dash)
################-----------------------##################

#### Alternate method is to first calculate the expected bounding box of the Warped I4 image and inverse map each pixel 
#___ of the bounding box to an tranformed pixel in I4 image. This ensures regularity in the Output Image.

# Bounding box limits calculation
bounds_I4 = [ [0,0,1], [0,I4.shape[0]-1,1], [I4.shape[1]-1,0,1], [I4.shape[1]-1,I4.shape[0]-1,1] ]
Xs, Ys = [], []
H_inv = np.linalg.inv(H_34)
for point in bounds_I4:
    v_dash = H_inv.dot(point) # Warping of the boundary points
    v_dash /= v_dash[2]
    x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
    Xs.append(x_dash)
    Ys.append(y_dash)
    
# Find reverse mapping for all pixels lying in Min to Max limits:
width_limits = range(max(min(Xs),-w_offset), min(max(Xs)+1,w_max-w_offset))
height_limits = range(max(min(Ys), -h_offset), min(max(Ys)+1,h_max-h_offset))

for y in tqdm(height_limits):
    for x in width_limits:
        v_dash = H_34.dot([x, y, 1]) # Reverse map to Image I4
        v_dash = v_dash/v_dash[2]
        x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
        
        # Important Condition to check: the inverse-mapped pixel should lie in I4:
        if(x_dash >= 0 and x_dash < I4.shape[1] and y_dash >= 0 and y_dash < I4.shape[0] ): 
            temp_image[y + h_offset, x + w_offset,:] = I4[y_dash,x_dash,:]
        if(x + w_offset >= x_min): # Calculating the Intersection Area for Blending
            common_area_xs.append(x)

try:    
    minn, maxx = min(common_area_xs), max(common_area_xs) # Intersection Region
    w_intersection = maxx - minn # Intersection width
    split_value = w_offset + (minn + maxx)//2 # Intersection position

    # Here, A & B are the sub images that are positionally warped and are now to be blended together!
    A = np.zeros(stitched_image.shape) 
    A[:,:split_value + w_intersection//2,:] = temp_image[:,:split_value + w_intersection//2,:]
    B = np.zeros(temp_image.shape)
    B[:,split_value - w_intersection//2 :,:] = stitched_image[:,split_value - w_intersection//2:,:]

    # Using the Pyramid based Blending method
    n_pyrs = 4 # Going deeper upto 4 levels for better blending

    # A mask is created for the warped image so that we know which portion to use for blending in which area.
    mask = np.zeros(temp_image.shape)
    mask[:,:split_value,:] = 1

    # Gaussian pyramids for all 4 levels to be found as each level will have own poition of splitting
    mask_pyrs = gaussian_pyr(mask, n_pyrs)
#     cv2.imwrite("blend_A.jpg",A)
#     cv2.imwrite("blend_B.jpg",B)

    # Laplacian pyramids of the two sub images to be blended
    LPA = laplacian_pyr(A, n_pyrs)
    LPB = laplacian_pyr(B, n_pyrs)

    blended = blending_pyrs(LPA, LPB, mask_pyrs) # Blending of these LPs according to the masks at each level
    result = image_reconstruction(blended) # Reconstruction of the Image back to higher resolution levels

    cv2.imwrite("./results/I{}/stitch{}_1_2.jpg".format(Image_Dataset_No,flag),result)
    stitched_image = result
except Exception as e:
    print(e)
    pass

100%|████████████████████████████████████████████████████████████████████████████████| 628/628 [00:06<00:00, 96.80it/s]


In [733]:
# Stitching I2 to I3: [3rd Right Image to the base image]

temp_image = np.zeros((h_max,w_max,3)) # To store the Warped I2 Image
common_area_xs = [] # To find the width and position of the intersecting region

x_max = np.max(np.where(stitched_image != 0)[1]) # Maximum width location of the already stitched image

#_________ The commented method below is to find the Warped image I2 that can be stitched to I3 easily.
#_________ But this method isn't effective because it finds the transformed I2 using the H23 homography and does not
#_________ necessarily have integral pixel locations. Thus blank dots found in the output Image.

################-----------------------##################
# for y in range(I2.shape[0]):
#     for x in range(I2.shape[1]):
#         v_dash = H_23.dot([x,y,1])
#         v_dash /= v_dash[2]
#         x_dash, y_dash = int(v_dash[0]), int(v_dash[1])      
#         temp_image[y_dash+h_offset-1 : y_dash+h_offset+2, x_dash+w_offset-1 : x_dash+w_offset+2,:] = I2[y,x,:]
################-----------------------##################

#### Alternate method is to first calculate the expected bounding box of the Warped I2 image and inverse map each pixel 
#___ of the bounding box to an tranformed pixel in I2 image. This ensures regularity in the Output Image.

# Bounding box limits calculation
bounds_I2 = [ [0,0,1], [0,I2.shape[0]-1,1], [I2.shape[1]-1,0,1], [I2.shape[1]-1,I2.shape[0]-1,1] ]
Xs, Ys = [], []
for point in bounds_I2:
    v_dash = H_23.dot(point) # Warping of the boundary points
    v_dash /= v_dash[2]
    x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
    Xs.append(x_dash)
    Ys.append(y_dash)
# Find reverse mapping for all pixels lying in Min to Max limits:
width_limits = range(max(min(Xs),-w_offset), min(max(Xs)+1,w_max-w_offset))
height_limits = range(max(min(Ys), -h_offset), min(max(Ys)+1,h_max-h_offset))
H_inv = np.linalg.inv(H_23)
for y in tqdm(height_limits):
    for x in width_limits:
        v_dash = H_inv.dot([x, y, 1]) # Reverse map to Image I2
        v_dash = v_dash/v_dash[2]
        x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
        # Important Condition to check: the inverse-mapped pixel should lie in I2:
        if(x_dash >= 0 and x_dash < I2.shape[1] and y_dash >= 0 and y_dash < I2.shape[0] ):
            temp_image[y + h_offset, x + w_offset,:] = I2[y_dash,x_dash,:]
        if(0 <= x + w_offset < x_max):
            common_area_xs.append(x) # Calculating the Intersection Area for Blending

minn, maxx = min(common_area_xs), max(common_area_xs) # Intersection Region
w_intersection = maxx - minn # Intersection width
split_value = w_offset + (minn + maxx)//2 # Intersection position

# Here, A & B are the sub images that are positionally warped and are now to be blended together!
A = np.zeros(temp_image.shape)
A[:,split_value - w_intersection//2 :,:] = temp_image[:,split_value - w_intersection//2:,:]
B = np.zeros(stitched_image.shape)
B[:,:split_value + w_intersection//2,:] = stitched_image[:,:split_value + w_intersection//2,:]

# Using the Pyramid based Blending method
n_pyrs = 4 # Going deeper upto 4 levels for better blending

# A mask is created for the warped image so that we know which portion to use for blending in which area.
mask = np.zeros(temp_image.shape)
mask[:,split_value:,:] = 1

# Gaussian pyramids for all 4 levels to be found as each level will have own poition of splitting
mask_pyrs = gaussian_pyr(mask, n_pyrs)
# cv2.imwrite("blend_A.jpg",A)
# cv2.imwrite("blend_B.jpg",B)
# Laplacian pyramids of the two sub images to be blended
LPA = laplacian_pyr(A, n_pyrs)
LPB = laplacian_pyr(B, n_pyrs)

blended = blending_pyrs(LPA, LPB, mask_pyrs) # Blending of these LPs according to the masks at each level
result = image_reconstruction(blended) # Reconstruction of the Image back to higher resolution levels
cv2.imwrite("./results/I{}/stitch{}_1_2_3.jpg".format(Image_Dataset_No,flag),result)
stitched_image = result

100%|███████████████████████████████████████████████████████████████████████████████| 579/579 [00:03<00:00, 157.12it/s]


In [734]:
# Stitching I1 to I3: [Farthest image to the base image]

temp_image = np.zeros((h_max,w_max,3)) # To store the Warped I1 Image
common_area_xs = [] # To find the width and position of the intersecting region

x_max = np.max(np.where(stitched_image != 0)[1]) # Maximum width location of the already stitched image

#_________ The commented method below is to find the Warped image I2 that can be stitched to I3 easily.
#_________ But this method isn't effective because it finds the transformed I2 using the H23 homography and does not
#_________ necessarily have integral pixel locations. Thus blank dots found in the output Image.

################-----------------------##################
# for y in range(I1.shape[0]):
#     for x in range(I1.shape[1]):
#         v_dash = ((H_12).dot(H_23)).dot([x,y,1])
#         v_dash /= v_dash[2]
#         x_dash, y_dash = int(v_dash[0]), int(v_dash[1])         
#         temp_image[y_dash+h_offset-2 : y_dash+h_offset+3, x_dash+w_offset-2 : x_dash+w_offset+3,:] = I1[y,x,:]
################-----------------------##################

#### Alternate method is to first calculate the expected bounding box of the Warped I1 image and inverse map each pixel 
#___ of the bounding box to an tranformed pixel in I1 image. This ensures regularity in the Output Image.

# Bounding box limits calculation
bounds_I1 = [ [0,0,1], [0,I1.shape[0]-1,1], [I1.shape[1]-1,0,1], [I1.shape[1]-1,I1.shape[0]-1,1] ]
Xs, Ys = [], []
HH = (H_12).dot(H_23)
for point in bounds_I1:
    v_dash = HH.dot(point) # Warping of the boundary points
    v_dash = v_dash/v_dash[2]
    x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
    Xs.append(x_dash)
    Ys.append(y_dash)
# Find reverse mapping for all pixels lying in Min to Max limits:
width_limits = range(max(min(Xs),-w_offset), min(max(Xs)+1,w_max-w_offset))
height_limits = range(max(min(Ys), -h_offset), min(max(Ys)+1,h_max-h_offset))

H_inv = np.linalg.inv((H_12).dot(H_23))
for y in tqdm(height_limits):
    for x in width_limits:
        v_dash = H_inv.dot([x, y, 1]) # Reverse map to Image I1
        v_dash = v_dash/v_dash[2]
        x_dash, y_dash = int(v_dash[0]), int(v_dash[1])
        # Important Condition to check: the inverse-mapped pixel should lie in I1:
        if(x_dash >= 0 and x_dash < I1.shape[1] and y_dash >= 0 and y_dash < I1.shape[0] and x + w_offset < w_max and y + h_offset < h_max):
            temp_image[y + h_offset, x + w_offset,:] = I1[y_dash,x_dash,:]
        if(0 <= x + w_offset < x_max):
            common_area_xs.append(x)
            
minn, maxx = min(common_area_xs), max(common_area_xs) # Intersection Region
w_intersection = maxx - minn # Intersection width
split_value = w_offset + (minn + maxx)//2 # Intersection position

# Here, A & B are the sub images that are positionally warped and are now to be blended together!
A = np.zeros(temp_image.shape)
A[:,split_value - w_intersection//2 :,:] = temp_image[:,split_value - w_intersection//2:,:]
B = np.zeros(stitched_image.shape)
B[:,:split_value + w_intersection//2,:] = stitched_image[:,:split_value + w_intersection//2,:]

# Using the Pyramid based Blending method
n_pyrs = 4 # Going deeper upto 4 levels for better blending

# A mask is created for the warped image so that we know which portion to use for blending in which area.
mask = np.zeros(temp_image.shape)
mask[:,split_value:,:] = 1

# Gaussian pyramids for all 4 levels to be found as each level will have own poition of splitting
mask_pyrs = gaussian_pyr(mask, n_pyrs)
# cv2.imwrite("blend_A.jpg",A)
# cv2.imwrite("blend_B.jpg",B)

# Laplacian pyramids of the two sub images to be blended
LPA = laplacian_pyr(A, n_pyrs)
LPB = laplacian_pyr(B, n_pyrs)

blended = blending_pyrs(LPA, LPB, mask_pyrs) # Blending of these LPs according to the masks at each level
result = image_reconstruction(blended) # Reconstruction of the Image back to higher resolution levels
cv2.imwrite("./results/I{}/Panorama{}_1_2_3_4.jpg".format(Image_Dataset_No,flag),result)
stitched_image = result

100%|███████████████████████████████████████████████████████████████████████████████| 646/646 [00:04<00:00, 130.79it/s]
